In [297]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode
from datetime import datetime
import pickle
# from torch.utils.tensorboard import SummaryWriter


# husk undersøgelser
forskellige modeller

# test af hyperparametre
blandede variabler

# mål accuraccy
loss og accuracy

In [298]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [299]:
# get words
all_words = []


with open('./data/unique_words_2k_articles.dat', 'rb') as file:
    all_words = pickle.load(file)

vocab_length = len(all_words)
print(f'number of characters: {vocab_length}')


number of characters: 20448


In [300]:
# read file
corpus = []
with open('./data/tokenized_articles.dat', 'rb') as file:
    corpus = pickle.load(file)
print(corpus[0])


['defense', '<quotation_mark>', 'discretionary', '<quotation_mark>', 'spending', 'on', 'programs', 'that', 'support', 'education', '<comma>', 'scientific', 'research', '<comma>', 'infrastructure', '<comma>', 'public', 'health', 'and', 'environmental', 'protection', '<period>', '<quotation_mark>', 'the', '<left_paren>', 'trump', '<right_paren>', 'administration', 'has', 'already', 'been', 'willing', 'to', 'say', '<colon>', 'were', 'going', 'to', 'increase', 'non', '<question_mark>', 'defense', 'discretionary', 'spending', '<period>', '<period>', '<period>', 'by', 'about', '7', 'percent', '<comma>', '<quotation_mark>', 'meadows', '<comma>', 'chairman', 'of', 'the', 'small', 'but', 'influential', 'house', 'freedom', 'caucus', '<comma>', 'said', 'on', 'the', 'program', '<period>', '<quotation_mark>', 'now', '<comma>', 'democrats', 'are', 'saying', 'thats', 'not', 'enough', '<comma>', 'we', 'need', 'to', 'give', 'the', 'government', 'a', 'pay', 'raise', 'of', '10', 'to', '11', 'percent', '<

In [301]:
# module

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        print(f'input_size: {input_size}')
        self.embed = nn.Linear(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(self.hidden_size * num_layers, output_size)

    def forward(self, x, hidden, cell):
        print(f'x: {x}')
        print(f'x.shape:')
        print(x.shape)
        print(hidden.shape)
        print(cell.shape)
        out = self.embed(x)
        print('here?')
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        return hidden, cell



In [302]:
class Generator():
    def __init__(self, word2vec, vec2word):
        self.word2vec = word2vec
        self.vec2word = vec2word
        self.chunk_len = 20
        self.num_epochs = 4000
        self.batch_size = 1
        self.print_every = self.num_epochs // 25 or 1
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003

    # returns tensor for entire sentence
    def word_tensor(self, word_sequence):
        tensor = torch.zeros(len(word_sequence)*2).float()
        # print(f'word_sequence: {word_sequence}')
        word = word_sequence[0][0]
        # print(f'word: {word}')
        for c in range(0, len(word_sequence), 2):
            word = word_sequence[0]
            # print(f'word2vec: {self.word2vec[word_sequence[c]]}')
            tensor[c], tensor[c+1] = self.word2vec[word_sequence[0][c]]
        return tensor


    def get_random_batch(self):
        start_idx = random.randint(0, len(corpus) - self.chunk_len)
        end_idx = start_idx + self.chunk_len + 1
        text_str = corpus[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len * 2)
        text_target = torch.zeros(self.batch_size, self.chunk_len * 2)
        for i in range(self.batch_size):
            text_input[i,:] = self.word_tensor(text_str[:-1])
            text_target[i,:] = self.word_tensor(text_str[1:])
        
        return text_input.float(), text_target.float()


    def generate(self, initial_words=['i', 'would', 'like'], predict_len=200, temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_words)
        predicted = initial_words
        
        for p in range(len(initial_words) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)

        last_word = initial_input[-1]
        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_word.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_word = torch.multinomial(output_dist, 1)[0]
            predicted_word = all_words[top_word]
            predicted += predicted_word
            print(predicted)
            last_word = self.char_tensor(predicted_word)

        return predicted


    def train(self):
        self.rnn = RNN(vocab_length, self.hidden_size,
                       self.num_layers, vocab_length).to(device)
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        print(f'<{datetime.now()}>starting training')

        for epoch in range(1, self.num_epochs + 1):
            input, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)

            self.rnn.zero_grad()
            loss = 0
            input = input.to(device)
            target = target.to(device)

            for c in range(0, self.chunk_len, 2):
                x = torch.zeros(2).float().to(device)
                print('-'*40)
                print('x.shape')
                print(x.shape)
                print('input.shape')
                print(input.shape)
                print(input[:, 0:1])

                print('input')

                print(input)
                x[0] = input[:,:1]
                x[1] = input[:,:-1]
                print(x)
                print(x.shape)
                print('-'*40)
                output, (hidden, cell) = self.rnn(x, hidden, cell)
                loss += criterion(output, target[:, c])

            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f'\n\n<{datetime.now()}> | epoch: {epoch}/{self.num_epochs} | loss: {loss}')
                print(self.generate())


In [303]:
lookup_w2v = {}
lookup_v2w = {}

with open('./models/w2v__10000_ep_2000_art_20448_words.dat', 'rb') as f_path:
    lookup_w2v = pickle.load(f_path)


with open('./models/v2w_10000_ep_2000_art_20448_words.dat', 'rb') as f_path:
    lookup_v2w = pickle.load(f_path)

x, y = lookup_w2v['trump']
trump = lookup_v2w[(x, y)]
print(trump)


trump


In [304]:


gentext = Generator(lookup_w2v, lookup_v2w)

gentext.train()


input_size: 20448
<2023-05-29 11:22:40.333169>starting training
----------------------------------------
x.shape
torch.Size([2])
input.shape
torch.Size([1, 40])
tensor([[-0.5665]], device='cuda:0')
input
tensor([[-0.5665,  0.5179, -1.2829,  0.6886, -0.4476, -0.1006,  0.4528, -0.6932,
         -1.5671,  0.3445, -0.6784, -0.6908, -0.6283,  1.1758, -0.5382, -0.0641,
         -0.7157,  0.7051, -1.1606,  0.6913,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')


RuntimeError: expand(torch.cuda.FloatTensor{[39]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)

In [ ]:
# gentext.generate(initial_str='i would like', predict_len=400, temperature=0.65)


In [ ]:
# m = nn.Linear(512, 100)
# input = torch.randn(1, 512)
# output = m(input)
# print(output)
